In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import seaborn as sns
from itertools import permutations, combinations

In [ ]:
sns.set_style("darkgrid")

In [ ]:
df = pd.read_csv('../dataset/clean/sample_sorted.csv', header=0)

In [ ]:
# Perform leadlag for every combination of features across all participants (LONG)
pairs = list(combinations(df.columns, 2))
upper = 14
corr_range = list(range(-upper,upper+1,1))
# Every possible feature combination has leadlag computed for it. PLotted based on
# the delay used to shift one signal. Participants are grouped and then concatenated
# so everything matches up
for x, y in pairs:
    if (df[x].dtype == object) or (df[y].dtype == object) or x == 'pid' or y == 'pid': continue
    pid_dfs = []
    shift_dfs = []
    for pid, pid_df in df.groupby('pid'):
        pid_dfs.append(pid_df.sort_values('date')[x])
        shift_dfs.append(pid_df.sort_values('date')[y])
    
    # Shift all participant dataframes, concatenate, and compute correlations
    base_df = pd.concat(pid_dfs, axis=0, ignore_index=True)
    cors = []
    for i in corr_range:
        outer_df = pd.concat([s_df.shift(i) for s_df in shift_dfs], axis=0, ignore_index=True)
        cors.append(base_df.corr(outer_df))
    
    # Plot
    plt.figure(figsize=(12,10))
    sns.barplot(x=corr_range, y=cors, color='blue')
    plt.title(f'{x}-{y}')
    plt.show()
    plt.close()      